In [1]:
import sys
sys.path.append('..')
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing
from QCreator.auxiliary_functions import *

tl_core = 10.
tl_gap = 6.
tl_ground = 10.


# resonator_core = 7
# resonator_gap = 4
# resonator_ground = 16
resonator_core = 8
resonator_gap = 7
resonator_ground = 16

pad_offset = 800
pad_element_offset = 1000
qubit_position_offset = 800

coupler_start = pad_offset + pad_element_offset
coupler_delta = 500
coupler_length = 450
num_couplers = 1

In [2]:
reload(gdspy)
reload(creator)
reload(elements)


layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':1,
    'air bridges':2,
    'air bridge pads':3,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':17
}

sample = creator.Sample('1Q_test',layers_configuration)

#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=4.7e3
sample.chip_geometry.sample_horizontal_size=4.7e3
# num_qubits_line = 3

central_line_y = sample.chip_geometry.sample_vertical_size/2

airbridge = elements.AirBridgeGeometry(pad_width = 5,
                                       pad_length = 4,
                                       bridge_width = 4,
                                       bridge_length = 38,
                                       pad_distance = 32,
                                       layer_configuration = sample.layer_configuration)



chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads)
sample.add(chip_edge_ground)

In [3]:
# 1. Create contact pads:
pads_left = []
pads_right = []
for pad_side_id in range(1):
    pad = elements.Pad('pad-left-'+str(pad_side_id), 
                       (pad_offset,  sample.chip_geometry.sample_vertical_size/2), np.pi, tl_core, 
                       tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_left.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-right-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size-pad_offset,  
                        sample.chip_geometry.sample_vertical_size/2), 0, tl_core, 
                       tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_right.append(pad)
    sample.add(pad)
    
pads_top = []
pads_bottom = []
for pad_side_id in range(3):
    pad = elements.Pad('pad-bottom-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size/4*(pad_side_id+1), pad_offset), 
                       -np.pi/2, tl_core, tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_bottom.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-top-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size/4*(pad_side_id+1),
                        sample.chip_geometry.sample_vertical_size-pad_offset), 
                        np.pi/2, tl_core, tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_top.append(pad)
    sample.add(pad)
    
p1 = pads_left[0]
p2 = pads_right[0]

In [4]:
g5, g6, res, open_part_of_res = draw_single_resonator(sample,
                                                      coupler_start, central_line_y, coupler_length,
                                                     resonator_core,resonator_gap,resonator_ground, 
                                                     tl_core, tl_gap, tl_ground, grounding_width=10,
                                                     closed_end_meander_length=3000,length_left=200,length_right=400,
                                                     open_end_legth=800, 
                                                     airbridge=airbridge, 
                                                     min_bridge_spacing=150,
                                                     port_orientation='right', direction_orientation='up')

cpw segments [{'type': 'endpoint', 'endpoint': array([1789., 2390.]), 'length': 0}, {'type': 'endpoint', 'endpoint': array([1789., 3190.]), 'length': 800.0, 'startpoint': array([1789., 2390.])}]
[CPW "right open ends0", l=156.000, CPW "right open ends1", l=156.000, CPW "right open ends2", l=156.000, CPW "right open ends3", l=156.000, CPW "right open ends4", l=156.000]


In [5]:
connector = sample.connect_cpw(o1=p1, o2=g5, port1='port', port2='narrow', name='---',
                    points=[], airbridge = airbridge, min_spacing=150)

cpw segments [{'type': 'endpoint', 'endpoint': array([ 800., 2350.]), 'length': 0}, {'type': 'segment', 'startpoint': array([ 800., 2350.]), 'endpoint': array([ 800.40993175, 2350.        ]), 'length': 0.40993175348921795}, {'type': 'turn', 'turn': -0.01967839867865836, 'startpoint': array([ 800.40993175, 2350.        ]), 'endpoint': array([ 802.06281056, 2349.98373647]), 'length': -1.6529854890073021, 'instead_point': array([ 801.23645117, 2350.        ])}, {'type': 'segment', 'startpoint': array([ 802.06281056, 2349.98373647]), 'endpoint': array([1765.81117603, 2331.01626353]), 'length': 963.9349962357322}, {'type': 'turn', 'turn': 0.019678398678658348, 'startpoint': array([1765.81117603, 2331.01626353]), 'endpoint': array([1767.46405483, 2331.        ]), 'length': 1.6529854890073012, 'instead_point': array([1766.63753542, 2331.        ])}, {'type': 'endpoint', 'endpoint': array([1768., 2331.]), 'length': 0.5359451655008343, 'startpoint': array([1767.46405483, 2331.        ])}]
[CPW 

In [6]:
connector = sample.connect_cpw(o1=g6, o2=p2, port1='narrow', port2='port', name='---',
                    points=[])

cpw segments [{'type': 'endpoint', 'endpoint': array([2282., 2331.]), 'length': 0}, {'type': 'segment', 'startpoint': array([2282., 2331.]), 'endpoint': array([2282.2461024, 2331.       ]), 'length': 0.24610239647381604}, {'type': 'turn', 'turn': 0.011754015301857695, 'startpoint': array([2282.2461024, 2331.       ]), 'endpoint': array([2283.23341695, 2331.00580252]), 'length': 0.9873372853560464, 'instead_point': array([2282.73977672, 2331.        ])}, {'type': 'segment', 'startpoint': array([2283.23341695, 2331.00580252]), 'endpoint': array([3898.64057793, 2349.99419748]), 'length': 1615.5187572072066}, {'type': 'turn', 'turn': -0.011754015301857797, 'startpoint': array([3898.64057793, 2349.99419748]), 'endpoint': array([3899.62789248, 2350.        ]), 'length': -0.9873372853560549, 'instead_point': array([3899.13421815, 2350.        ])}, {'type': 'endpoint', 'endpoint': array([3900., 2350.]), 'length': 0.37210752169175976, 'startpoint': array([3899.62789248, 2350.        ])}]


In [7]:
sample.draw_design()
sample.watch()

C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\gdspy\path.py:827: UserWarning: [GDSPY] Possible inconsistency found in `FlexPath` due to sharp corner.
  warnings.warn(


In [8]:
sample.write_to_gds()

Gds file has been writen here:  D:\Documents-cloud\qubits\simulation\QCreator\example_notebooks\1Q_test.gds


In [8]:
sys, connections, elements = sample.get_tls()

In [9]:
sample.get_topology()

[[Short , [0]],
 [Port pad-left-0, [43]],
 [Port pad-right-0, [64]],
 [Port pad-bottom-0, [65]],
 [Port pad-top-0, [66]],
 [Port pad-bottom-1, [67]],
 [Port pad-top-1, [68]],
 [Port pad-bottom-2, [69]],
 [Port pad-top-2, [70]],
 [TL TL-resonator coupler (n=3), [1, 2, 3, 6, 7, 8]],
 [TL closed end resonator fanout_line_0 (n=2), [71, 72, 4, 5]],
 [TL closed end resonator fanout_line_1 (n=1), [73, 29]],
 [TL closed end resonator fanout_coupled_line_2 (n=3), [1, 2, 3, 71, 72, 73]],
 [R cl10, [5, 0]],
 [TL cl1 (n=1), [4, 44]],
 [TL open end resonator fanout_line_0 (n=1), [74, 31]],
 [TL open end resonator fanout_line_1 (n=2), [75, 76, 9, 10]],
 [TL open end resonator fanout_coupled_line_2 (n=3), [8, 7, 6, 74, 75, 76]],
 [R cl20, [9, 0]],
 [TL cl2 (n=1), [10, 63]],
 [TL closed ends0 (n=1), [29, 11]],
 [TL closed endb0_line1 (n=1), [11, 77]],
 [TL closed endb0_line2 (n=1), [77, 12]],
 [C , [77, 0]],
 [TL closed ends1 (n=1), [12, 13]],
 [TL closed endb1_line1 (n=1), [13, 78]],
 [TL closed endb

In [10]:
omega, kappa, modes = sys.get_modes()

In [11]:
omega / (2* np.pi)

array([6.94488595e+09, 2.07913705e+10, 2.25858535e+10, 3.46359235e+10,
       4.42816214e+10, 4.85283019e+10, 6.24061944e+10, 6.50790817e+10,
       7.63000298e+10, 8.42434918e+10, 8.99617504e+10, 1.03682976e+11,
       1.03897329e+11, 1.17666371e+11, 1.22630833e+11, 1.24022852e+11,
       1.31806592e+11, 1.43552770e+11, 1.45621144e+11, 1.59450750e+11,
       1.62270690e+11, 1.73215503e+11, 1.82300672e+11, 1.86114230e+11,
       1.98206538e+11, 2.02357404e+11, 2.08655811e+11, 2.19130389e+11,
       2.21105504e+11, 2.31314036e+11, 2.40529236e+11, 2.44625734e+11,
       2.45831123e+11, 2.46234922e+11, 2.46234922e+11, 2.49468428e+11,
       2.49468428e+11, 2.60418036e+11, 2.61154830e+11, 2.76265236e+11,
       2.80562804e+11, 2.92062184e+11, 2.98984546e+11, 3.06869694e+11,
       3.18668252e+11, 3.20152302e+11, 3.31576264e+11, 3.38613722e+11,
       3.41928075e+11, 3.47225278e+11, 3.50283790e+11, 3.57849162e+11,
       3.63397340e+11, 3.65430740e+11, 3.73117269e+11, 3.82459647e+11,
      

In [ ]:
1.930694